In [16]:
import pandas as pd

# Read the oil price data
oil_df = pd.read_csv('oil.csv')

# Display the first few rows to understand its structure
print(oil_df.head())

         date  dcoilwtico
0  2013-01-01         NaN
1  2013-01-02       93.14
2  2013-01-03       92.97
3  2013-01-04       93.12
4  2013-01-07       93.20


處理oil.csv 中油價的缺失值，方法為缺失值前後一日的油價計算平均後填入
使用這個方法是因為，雖然大多數油價相差不大(在1元以內)，但是有時會相差達5(如2014/9/1)
因此比起簡單的前後填充，使用平均更加合理

最後執行一次backfill，處理第一個缺失值

In [17]:
#conver the date and oilprice to np data formate 
oil_df['date'] = pd.to_datetime(oil_df['date'], errors='coerce')  # Convert to datetime, invalid parsing will be set as NaT
oil_df['dcoilwtico'] = pd.to_numeric(oil_df['dcoilwtico'], errors='coerce')  # Convert to numeric, invalid parsing will be set as NaN

# Handle missing values
#use the bake value to fillup the mising value

def fillup_with_mean(series):
    filled_series = series.copy()
    forward_fill = filled_series.ffill()
    backwad_fill = filled_series.bfill()

    mean_fill = (forward_fill+backwad_fill)/2
    return series.fillna(mean_fill)


oil_df['dcoilwtico'] = fillup_with_mean(oil_df['dcoilwtico'])
oil_df['dcoilwtico'] = oil_df['dcoilwtico'].bfill()

print(oil_df.head(437))


          date  dcoilwtico
0   2013-01-01       93.14
1   2013-01-02       93.14
2   2013-01-03       92.97
3   2013-01-04       93.12
4   2013-01-07       93.20
..         ...         ...
432 2014-08-28       96.44
433 2014-08-29       97.86
434 2014-09-01       95.39
435 2014-09-02       92.92
436 2014-09-03       95.50

[437 rows x 2 columns]


 read the sales data and add the oil price column
 將兩者merge，以sales.csv的date欄位作為依據

In [18]:
sales_df = pd.read_csv('sales.csv')

#print(sales_df.head())

#ensure the date columns in both dataframs are in the datetime format for merging
sales_df['date'] = pd.to_datetime(sales_df['date'],errors = 'coerce')

merged_df = pd.merge(sales_df,oil_df[['date','dcoilwtico']],on = 'date',how = 'left')

print(merged_df.head())
print(merged_df.columns)


   id       date  store_nbr      family  sales  onpromotion  dcoilwtico
0   0 2013-01-01          1  AUTOMOTIVE    0.0            0       93.14
1   1 2013-01-01          1   BABY CARE    0.0            0       93.14
2   2 2013-01-01          1      BEAUTY    0.0            0       93.14
3   3 2013-01-01          1   BEVERAGES    0.0            0       93.14
4   4 2013-01-01          1       BOOKS    0.0            0       93.14
Index(['id', 'date', 'store_nbr', 'family', 'sales', 'onpromotion',
       'dcoilwtico'],
      dtype='object')


Save the Updated Sales Data

In [20]:
merged_df.to_csv('updated_sales.csv',index = False)

PermissionError: [Errno 13] Permission denied: 'updated_sales.csv'